<a href="https://colab.research.google.com/github/AshAninze/Uni-Projects/blob/main/Estimation_of_Obesity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The dataset was selected to be able to predict obesity levels based on the

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Postgrad/Comp4AI/ObesityDataSet_raw_and_data_sinthetic.csv')

df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [41]:
#importing all needed packages for ml pipeline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split, KFold, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from timeit import default_timer as timer
from sklearn.svm import SVC
from xgboost import XGBClassifier
GBC = XGBClassifier(n_estimators=100)
import warnings
warnings.filterwarnings("ignore") #ignoring any warnings

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

class DataSciencePipeline:
  ''' A class to create multiple machine learning model on a given dataset

      This class encapsulates the entire data science pipline, including:
      - Data Loading
      - Data preprocessing
      - Feature Engineering
      - Data splitting
      - Model training and evaulation
      '''

  def __init__(self):
    ''' Initializes the DataSciencePipeline with the given dataset.

        Attributes:
          data: The pandas DataFrame holding the dataset.
          X: Features (independent variables) after preprocessing.
          y: Target variable (dependent variable).
          no_X: Features without scaling, used for comparison.
          no_y: Target variable without scaling.
          X_train: Training features variables.
          X_test: Testing features varibles.
          y_train: Training target variable.
          y_test: Testing target variable.
           '''
    self.data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Postgrad/Comp4AI/ObesityDataSet_raw_and_data_sinthetic.csv')
    self.X = None
    self.y = None
    self.no_X = None
    self.no_y = None
    self.X_train = None
    self.X_test = None
    self.y_train = None
    self.y_test = None


  def preprocess_data(self, categorical_columns, numerical_columns):
        """
        Preprocesses the data by rounding any numerical values to the nearest integer
        and then performing label encoding on categorical features.

        Attributes:
          data['Height']: Height in m
          data[column]: Given column name in list

        Args:
            categorical_columns: A list of column names that represent categorical features.
            numerical_columns: A list of column names that represent numerical features.

        Returns: Label encoded dataset with numerical values rounded to the nearest integer.
        """
        self.data['Height'] = self.data['Height'].round(2) #normlise the height value to x.xx

        for column in numerical_columns:
            self.data[column] = self.data[column].round()

        for column in categorical_columns: #label encoding due to the large amount of catorical values, to reducing processing time
          self.data[column] = self.data[column].astype(str)
          self.data[column] = LabelEncoder().fit_transform(self.data[column])

        # updating self x and y after encoding while creating a seprate
        # attribute for scaling
        self.X = self.data.drop('NObeyesdad', axis=1)
        self.y = self.data['NObeyesdad']
        self.no_X = self.data.drop('NObeyesdad', axis=1)
        self.no_y = self.data['NObeyesdad']

        print(self.data.head()) #print the encoded dataset

  def feature_engineering(self):
    '''
    Creates a column called BMI which is the body mass index.

    BMI is callucalted by weight divided by height squared.

    Attributes:
      data['BMI']: Body Mass Index
      data['Weight']: Weight in kg
      data['Height']: Height in m
    '''
    self.data['BMI'] = self.data['Weight'] / (self.data['Height'] ** 2)

  def split_data(self, test_size=0.2, random_state=42):
        """
        Splits the data into training and testing sets.

        Args:
            test_size: The proportion of the dataset to include in the test split.
            random_state: Controls the shuffling applied to the data before applying the split.
        """
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=test_size, random_state=random_state
        )

  def scale_data(self): #seperate scaling function, to be able to call after splitting
    '''
    Scales the data using the StandardScaler.

    Data scaling is the process of ajusting the range of values in features to a
    common scale.
    '''
    self.no_X_train = self.X_train
    self.no_X_test = self.X_test
    sc = StandardScaler()
    self.X_train = sc.fit_transform(self.X_train)
    self.X_test = sc.transform(self.X_test)

  def hyperparameter_tuning(self,models):
    """
    Tunes the hyperparameters of a given model using RandomSearchCV.


    Attributes:
      model: The name of the machine learning model to tune.
      param_grid: A dictionary of hyperparameters and their possible values.
      tuned_models: A dictionary containing the best estimator and its accuracy.

    Args:
      models: The list of the machine learning model to tune.

    Returns:
      A dictionary containing the best estimator, its accuracy and time taken to train.
    """
    tuned_models = {} #empty dict for the tuned models
    param_grids = { #parameter ranges for each model
        SVC: {'C': uniform(0.1, 10), 'kernel': ['linear', 'rbf'], 'gamma': uniform(1e-4, 1e+1)},
        RandomForestClassifier: {'n_estimators': randint(50, 200), 'max_depth': [None, 5, 10], 'min_samples_split': randint(2, 10)},
        LogisticRegression: {'C': uniform(0.1, 10), 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']},
        DecisionTreeClassifier: {'max_depth': [None, 5, 10], 'min_samples_split': randint(2, 10)}
    }

    for model in models:#trains model in param_grid
        param_distributions = param_grids.get(type(model))# retrieve the parameter distribution for the model
        random_search = RandomizedSearchCV(
            model, param_distributions, n_iter=50, cv=5, scoring='accuracy', random_state=42, n_jobs=-1
        )
        start_time = timer()
        random_search.fit(self.X_train, self.y_train)
        model_time = timer() - start_time
        scores = cross_val_score(model, self.X, self.y, cv=5) #trains cross validation scores for model comparisions for over fitting
        average_score = scores.mean()
        best_model = random_search.best_estimator_
        acc = accuracy_score(self.y_test, best_model.predict(self.X_test))

        tuned_models[model.__class__.__name__] = {'model': best_model, 'accuracy': acc}

        # printing model details
        print(f"Model: {model.__class__.__name__}")
        print("Best hyperparameters:", random_search.best_params_)
        print(f"Accuracy: {acc*100:.2f}%\n")
        print(f"Average accuracy: {average_score*100:.2f}%\n")
        print(f"Timing: {model_time:.2f}s\n")

    return tuned_models

  def train(self, models):
        """
        Trains the given model on the training data.

        Attributes:
        model_name: The name of the machine learning model.
        y_pred: The predicted target variable.
        acc: The accuracy of the model.
        model_time: The time taken to train the model.

        Args:
            models: The machine learning models to train.
        """
        for model_name in models: # trains models on unscaled data to be able to compare the differences
          start_time = timer()
          model_name.fit(self.no_X_train, self.y_train)
          model_time = timer() - start_time
          self.y_pred = model_name.predict(self.no_X_test)
          acc = accuracy_score(self.y_test, self.y_pred)
          print(f" Model: {model_name} Accuracy with no scaling: {acc*100:.2f}% Timing: {model_time:.2f}s\n")

        for model_name in models:# trains models within a list
          start_time = timer()
          model_name.fit(self.X_train, self.y_train)
          model_time = timer() - start_time
          self.y_pred = model_name.predict(self.X_test)
          acc = accuracy_score(self.y_test, self.y_pred)
          print(f" Model: {model_name} Accuracy with scaling: {acc*100:.2f}% Timing: {model_time:.2f}s\n ")

  def feature_importance(self, models):
    for model_name in models:
      importances = model_name.fit(self.X_train, self.y_train).feature_importances_
      featureIm = pd.DataFrame({'Feature': self.X.columns, 'Importance': importances})
      plt.figure(figsize=(10, 6))
      plt.barh(featureIm['Feature'], featureIm['Importance'])
      plt.xlabel('Importance')
      plt.ylabel('Feature')
      plt.title('Feature Importances')
      plt.show()

def main():
  '''
  Main function to run the data science pipeline.
  '''
  models = [SVC(), RandomForestClassifier(),LogisticRegression(), DecisionTreeClassifier()]
  categorical_columns = ['Gender','family_history_with_overweight','FAVC','CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS','NObeyesdad']
  numerical_columns = ['Age','NCP','CH2O','FAF','TUE','BMI']
  pipeline = DataSciencePipeline()
  pipeline.feature_engineering()
  pipeline.preprocess_data(categorical_columns, numerical_columns)
  pipeline.split_data()
  pipeline.scale_data()
  pipeline.train(models)
  pipeline.hyperparameter_tuning(models)
  #pipeline.feature_importance(models = {RandomForestClassifier(), XGBClassifier()})

main()



   Gender   Age  Height  Weight  family_history_with_overweight  FAVC  FCVC  \
0       0  21.0    1.62    64.0                               1     0   2.0   
1       0  21.0    1.52    56.0                               1     0   3.0   
2       1  23.0    1.80    77.0                               1     0   2.0   
3       1  27.0    1.80    87.0                               0     0   3.0   
4       1  22.0    1.78    89.8                               0     0   2.0   

   NCP  CAEC  SMOKE  CH2O  SCC  FAF  TUE  CALC  MTRANS  NObeyesdad   BMI  
0  3.0     2      0   2.0    0  0.0  1.0     3       3           1  24.0  
1  3.0     2      1   3.0    1  3.0  0.0     2       3           1  24.0  
2  3.0     2      0   2.0    0  2.0  1.0     1       3           1  24.0  
3  3.0     2      0   2.0    0  2.0  0.0     1       4           5  27.0  
4  1.0     2      0   2.0    0  0.0  0.0     2       3           6  28.0  
 Model: SVC() Accuracy with no scaling: 74.47% Timing: 0.05s

 Model: Rando